In [96]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,WebDriverException
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError
import itertools
import multiprocessing
import Ipynb_importer
from IPpool import GetHeaders
from IPpool import Ip_pool
import random
import copy

In [104]:
#**************将装机地址变成一组一组*****************
def get_chunk(adr_ls,n):
    new_adr_ls = copy.deepcopy(adr_ls)
    adr_chunk = [new_adr_ls[i:i + n] for i in range(0, len(new_adr_ls), n)]
    return adr_chunk

In [27]:

#设置谷歌浏览器的参数
def set_chrome_param():
    #PROXY = '175.155.24.61:808'
    options = webdriver.ChromeOptions()
    chrome_path = '/home/zhh/下载/chromedriver'  
    #options.add_argument('--proxy-server={0}'.format(PROXY))
    # 设置成中文
    options.add_argument('lang=zh_CN.UTF-8')
    # 添加头部
    #1允许所有图片；2阻止所有图片；3阻止第三方服务器图片
    prefs = {'profile.default_content_setting_values': { 'images':2}}
    options.add_experimental_option('prefs', prefs)
    options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"')
    chrome = webdriver.Chrome(chrome_path,chrome_options=options)
    return chrome


In [28]:
#requests请求网页，返回一个BeautifulSoup对象
def requests_visit_url(url,prox_ip=None):
    headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":"http://www.dianping.com/shop/92374637",
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}
    headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
    #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"}
    is_continue = True
    while is_continue:
        try:
            if prox_ip is None:
                res = requests.get(url,headers = headers)  #访问url，不设代理ip访问
            else:
                http = prox_ip[0]
                ip = http+"://"+prox_ip[1]
                proxies={http:ip}
                res = requests.get(url,headers = headers,proxies = proxies) #设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            is_continue = False 
            
        except HTTPError as e:
            print("断网")
            time.sleep(2)
            
        except Timeout as e:
            print("请求超时")
            time.sleep(2)
            
        except ConnectionError as e:
            print("访问被拒")
            is_continue = True
            time.sleep(2)
    return soup

In [29]:
#缓存结果
def save_cache(result,filename):
    with open(filename,'wb') as f1:
        pickle.dump(result,f1)

In [30]:
#加载缓存
def read_cache(filename):
    with open(filename,'rb')as f1:
        result = pickle.load(f1)
    return result

In [31]:
# 切换路径
def get_spc_path(father_path,child_path):
    os.chdir(father_path)
    if os.path.exists(child_path):
        os.chdir(child_path)
    else:
        os.mkdir(child_path)
        os.chdir(child_path)
    print("done!!")

## 1.二级分类层

In [32]:
%%time
#获取所有的二级分类url
def classify(city_url,city,big_class):
    soup = requests_visit_url(city_url)
    clf_ls = []
    for i in soup.select("#classfy a"):
        clf_url = i["href"] #分类网址
        clf_tl = i.text.strip() # 分类标题
        clf_ls.append([big_class,city,clf_tl,clf_url])
    return clf_ls

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.2 µs


In [41]:
%%time
clf_ls = classify("http://www.dianping.com/search/category/7/10",'深圳',"美食")

CPU times: user 180 ms, sys: 0 ns, total: 180 ms
Wall time: 1.63 s


In [42]:
clf_ls

[['美食', '深圳', '面包甜点', 'http://www.dianping.com/search/category/7/10/g117'],
 ['美食', '深圳', '小吃快餐', 'http://www.dianping.com/search/category/7/10/g112'],
 ['美食', '深圳', '粤菜', 'http://www.dianping.com/search/category/7/10/g103'],
 ['美食', '深圳', '自助餐', 'http://www.dianping.com/search/category/7/10/g111'],
 ['美食', '深圳', '咖啡厅', 'http://www.dianping.com/search/category/7/10/g132'],
 ['美食', '深圳', '日本料理', 'http://www.dianping.com/search/category/7/10/g113'],
 ['美食', '深圳', '火锅', 'http://www.dianping.com/search/category/7/10/g110'],
 ['美食', '深圳', '西餐', 'http://www.dianping.com/search/category/7/10/g116'],
 ['美食', '深圳', '小龙虾', 'http://www.dianping.com/search/category/7/10/g219'],
 ['美食', '深圳', '海鲜', 'http://www.dianping.com/search/category/7/10/g251'],
 ['美食', '深圳', '烧烤', 'http://www.dianping.com/search/category/7/10/g508'],
 ['美食', '深圳', '韩国料理', 'http://www.dianping.com/search/category/7/10/g114'],
 ['美食', '深圳', '川菜', 'http://www.dianping.com/search/category/7/10/g102'],
 ['美食', '深圳', '东南亚菜', 'http

## 2.区县层

In [52]:
def get_region(class_ls=None,prox_ip=None):
    print('正在爬取{0}'.format(class_ls[-2]))
    url = class_ls[-1]  #url
    soup = requests_visit_url(url,prox_ip) #访问
    region_ls = [] #地区的列表
    for i in soup.select("#region-nav a"):
        if i.text.strip() != "不限":
            region_url = i["href"] 
            region_tl = i.text.strip()
            region = class_ls[:-1]
            region.extend([region_tl,region_url])
            region_ls.append(region)
    return region_ls


In [50]:
ip = Ip_pool(['http://www.xicidaili.com/nn/'])
ip_pool = ip.get_ip_pool(test_url='http://www.dianping.com/search/category/7/10/g104',timeout =1)

http://www.xicidaili.com/nn/
{'Accept': 'application/json, text/javascript, */*; q=0.01', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'zh-CN,zh;q=0.8', 'Connection': 'keep-alive', 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3'}
坏ip: ['https', 'https://39.64.0.88:8118']
坏ip: ['http', 'http://144.12.24.3:52315']
坏ip: ['http', 'http://110.73.5.233:8123']
坏ip: ['https', 'https://27.151.11.211:80']
坏ip: ['https', 'https://27.150.85.171:808']
坏ip: ['http', 'http://182.246.207.192:80']
坏ip: ['https', 'https://61.159.195.6:8118']
坏ip: ['https', 'https://61.174.156.70:808']
坏ip: ['https', 'https://183.144.219.26:808']
坏ip: ['http', 'http://111.155.116.239:8123']
坏ip: ['http', 'http://61.178.238.122:63000']
坏ip: ['https', 'https://117.68.193.183:808']
坏ip: ['http', 'http://117.86.10.185:8888']
坏ip: ['http', 'http://121.12.42.176:61234']
坏ip: ['htt

In [54]:
%%time
prox_ip = random.choice(ip_pool)
region_ls = []
for i in clf_ls:
    print(i)
    sub_region = get_region(i,prox_ip)
    region_ls.append(sub_region)

['美食', '深圳', '面包甜点', 'http://www.dianping.com/search/category/7/10/g117']
正在爬取面包甜点
['美食', '深圳', '小吃快餐', 'http://www.dianping.com/search/category/7/10/g112']
正在爬取小吃快餐
['美食', '深圳', '粤菜', 'http://www.dianping.com/search/category/7/10/g103']
正在爬取粤菜
['美食', '深圳', '自助餐', 'http://www.dianping.com/search/category/7/10/g111']
正在爬取自助餐
['美食', '深圳', '咖啡厅', 'http://www.dianping.com/search/category/7/10/g132']
正在爬取咖啡厅
['美食', '深圳', '日本料理', 'http://www.dianping.com/search/category/7/10/g113']
正在爬取日本料理
['美食', '深圳', '火锅', 'http://www.dianping.com/search/category/7/10/g110']
正在爬取火锅
['美食', '深圳', '西餐', 'http://www.dianping.com/search/category/7/10/g116']
正在爬取西餐
['美食', '深圳', '小龙虾', 'http://www.dianping.com/search/category/7/10/g219']
正在爬取小龙虾
['美食', '深圳', '海鲜', 'http://www.dianping.com/search/category/7/10/g251']
正在爬取海鲜
['美食', '深圳', '烧烤', 'http://www.dianping.com/search/category/7/10/g508']
正在爬取烧烤
['美食', '深圳', '韩国料理', 'http://www.dianping.com/search/category/7/10/g114']
正在爬取韩国料理
['美食', '深圳', '川菜', 'http://www

In [55]:
region_ls

[[['美食',
   '深圳',
   '面包甜点',
   '福田区',
   'http://www.dianping.com/search/category/7/10/g117r29'],
  ['美食',
   '深圳',
   '面包甜点',
   '南山区',
   'http://www.dianping.com/search/category/7/10/g117r31'],
  ['美食',
   '深圳',
   '面包甜点',
   '罗湖区',
   'http://www.dianping.com/search/category/7/10/g117r30'],
  ['美食',
   '深圳',
   '面包甜点',
   '宝安区',
   'http://www.dianping.com/search/category/7/10/g117r33'],
  ['美食',
   '深圳',
   '面包甜点',
   '龙岗区',
   'http://www.dianping.com/search/category/7/10/g117r34'],
  ['美食',
   '深圳',
   '面包甜点',
   '龙华新区',
   'http://www.dianping.com/search/category/7/10/g117r12033'],
  ['美食',
   '深圳',
   '面包甜点',
   '盐田区',
   'http://www.dianping.com/search/category/7/10/g117r32'],
  ['美食',
   '深圳',
   '面包甜点',
   '大鹏新区',
   'http://www.dianping.com/search/category/7/10/g117r12036'],
  ['美食',
   '深圳',
   '面包甜点',
   '光明新区',
   'http://www.dianping.com/search/category/7/10/g117r12034'],
  ['美食',
   '深圳',
   '面包甜点',
   '坪山新区',
   'http://www.dianping.com/search/category/7/10/g117r120

## 3.商圈层

In [74]:
def get_circle(region_ls=None):
    url = region_ls[-1]   #url网址
    soup = requests_visit_url(url) #访问BeautifulSoup
    circle_ls =[]             
    for i in soup.select("#region-nav-sub a"):  #商圈信息
        if i.span.text.strip()!="不限":   #剔除掉不限的url
            circle_url = i["href"]       #商圈url
            circle_tit = i.span.text.strip()  #商圈名
            circle = region_ls[:-1] 
            circle.extend([circle_tit,circle_url]) 
            circle_ls.append(circle)
            
    #如果没有商圈的话，就拿原来的url当作是商圈的url
    if circle_ls==[]:
        print(region_ls[-2]+"没有子分类！！")
        region_ls.insert(4,"没有商圈")
        circle_ls.append(region_ls)

    return circle_ls
    

In [57]:
%%time
circle_ls = get_circle(['美食',
  '广州',
  '茶餐厅',
  '天河区',
  'http://www.dianping.com/search/category/4/10/g207r22'])

Wall time: 1.34 s


In [58]:
circle_ls

[['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '珠江新城',
  'http://www.dianping.com/search/category/4/10/g207r1522'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '石牌/龙口',
  'http://www.dianping.com/search/category/4/10/g207r1521'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河北',
  'http://www.dianping.com/search/category/4/10/g207r1520'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '高德置地/花城汇',
  'http://www.dianping.com/search/category/4/10/g207r30509'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '兴盛路/跑马场',
  'http://www.dianping.com/search/category/4/10/g207r30510'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河公园',
  'http://www.dianping.com/search/category/4/10/g207r1523'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '车陂/东圃',
  'http://www.dianping.com/search/category/4/10/g207r2080'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '龙洞/岑村',
  'http://www.dianping.com/search/category/4/10/g207r2598'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '燕岭/

## 4.商家信息层

In [75]:
#获取每个商家的链接、店名
def get_stores(circle_ls = None):
    store_ls=[] #存储的列表
    url = circle_ls[-1]  #url
    soup = requests_visit_url(url)  #访问当前主页
    is_continue = True
    page = 0
    while is_continue:
        page += 1
        print("正在爬取第{0}页".format(page))
        
        #解析数据
        if soup.select(".content") ==[]:
            print("此页没有任何信息！！")
            store_ls = []
            is_continue = False 
            
        else:
            for i in soup.select(".content"):  
                for j in i.select(".pic"):
                    store_name = j.img["alt"]   #店名
                    link = j.a["href"]     #详情层
                    store = circle_ls[:5]  #只取前5位
                    store.extend([url,store_name,link]) 
                    store_ls.append(store)


            #获取下一页的数据
            time.sleep(2)

            next_tag = soup.find_all("a",text = re.compile("下一页"))  #获取字符属性为下一页的url
            if next_tag != []:                                         #保证含有字符属性
                url = next_tag[0]["href"]                              #获取下一页的url
                soup = requests_visit_url(url)                         #更新BeautifulSoup对象
                time.sleep(2)
            else:
                url = ''
                is_continue  = False                                  #如果没有下一页，就停止更新
    
    return store_ls
    
#     #评论数据
#     for j in i.select(".comment"):

#         review_num =j.select(".review-num b")[0].text  #评论人数
#         mean_price = j.select(".mean-price b")[0].text #人均价格
        
#     for j in i.select(".addr"):
#         print("*******")
#         address = j.text  #地址
#         print("***********")

In [59]:
%%time
store_ls = get_stores(['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519'])

正在爬取第1页
正在爬取第2页
Wall time: 13.1 s


In [60]:
store_ls

[['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519',
  '港丽餐厅(天河城店)',
  'http://www.dianping.com/shop/6139793'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519',
  '石板街Pottinger',
  'http://www.dianping.com/shop/66875682'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519',
  '悦翠堂(太古汇店)',
  'http://www.dianping.com/shop/22203903'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519',
  '绿茶餐厅(天河城店)',
  'http://www.dianping.com/shop/45260813'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519',
  '翠华餐厅(正佳广场店)',
  'http://www.dianping.com/shop/15861615'],
 ['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519',
  '九龙冰室(正佳广场店)',
  'http://www.dianping.com/shop/20

## 5.详情层

In [ ]:
soup = requests_visit_url('http://www.dianping.com/shop/67814247')

In [76]:
#获取详情层的数据，如星级用户、浏览人数、均价
def get_info(store_ls=None):
    
    #首先先保证导入的数据不为空
    if store_ls !=[]:
        url = store_ls[-1] #详情信息表
        soup = requests_visit_url(url)
        try:
            start = soup.select(".mid-rank-stars")[0]["title"]       #星级用户
        except Exception as e:
            print("start",e)
            start = ''

        try:
            review_count = soup.select("#reviewCount")[0].text  #浏览人数
        except Exception as e:
            review_count = ''
            print("review_coun",e)

        try:
            avgprice = soup.select("#avgPriceTitle")[0].text   #均价
        except Exception as e:
            avgprice = ''
            print("avgprice",e)

        #评分系统
        try:
            score= []
            for j in soup.select("#comment_score .item"):
                score.append(j.text)
            score = "|".join(score)

        except Exception as e:
            score = ""
            print("score",e)

        try:   
            address = soup.select(".address .item")[0].text.strip() #地址
        except Exception as e:
            address = ''
            print("address出错了",e)

        #电话号码
        try:
            tel =[]
            for i in soup.select(".tel .item"):
                tel.append(i.text)
            tel = "|".join(tel)

        except:
            tel=""
            print("tel出错了",e)

        #路径
        try:
            path = []
            for i in soup.select(".breadcrumb a"):
                path.append(i.text.strip())
            path ='>'.join(path)
        except Exception as e:
            print("path出错了",e)
            path =''

        info = store_ls[:]
        info.extend([start,review_count,avgprice,score,address,tel,path])
    return info


In [64]:
get_info(['美食',
  '广州',
  '茶餐厅',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/10/g207r1519p2',
  '粤港烧卤鸭饭(天河店)',
  'http://www.dianping.com/shop/94311152'])

['美食',
 '广州',
 '茶餐厅',
 '天河区',
 '天河城/体育中心',
 'http://www.dianping.com/search/category/4/10/g207r1519p2',
 '粤港烧卤鸭饭(天河店)',
 'http://www.dianping.com/shop/94311152',
 '该商户暂无星级',
 '0条评论',
 '人均：-',
 '口味：0.0|环境：0.0|服务：0.0',
 '天河路190号首层2号',
 '13430394498',
 '广州餐厅>天河城/体育中心>茶餐厅']

## 6 主程序

In [71]:
%%time

clf_ls = classify("http://www.dianping.com/search/category/7/10",'深圳',"美食")
print("所有二级分类已经获取完毕")

region_ls = list(map(get_region,clf_ls))
print("获取所有的区域清单") 

所有二级分类已经获取完毕
正在爬取面包甜点
正在爬取小吃快餐
正在爬取粤菜
正在爬取自助餐
正在爬取咖啡厅
正在爬取日本料理
正在爬取火锅
正在爬取西餐
正在爬取小龙虾
正在爬取海鲜
正在爬取烧烤
正在爬取韩国料理
正在爬取川菜
正在爬取东南亚菜
正在爬取湘菜
正在爬取茶餐厅
正在爬取素菜
正在爬取江浙菜
正在爬取东北菜
正在爬取创意菜
正在爬取新疆菜
正在爬取西北菜
正在爬取粥粉面
正在爬取私房菜
正在爬取台湾菜
正在爬取农家菜
正在爬取其他
正在爬取家常菜
获取所有的区域清单
CPU times: user 5.62 s, sys: 208 ms, total: 5.82 s
Wall time: 1min 10s


In [72]:
#解链
region_ls = list(itertools.chain.from_iterable(region_ls))

In [78]:
%%time
#***************获取所有的商圈信息****************************888
circle_ls = list(map(get_circle,region_ls))

光明新区没有子分类！！
光明新区没有子分类！！
CPU times: user 51.9 s, sys: 1.57 s, total: 53.5 s
Wall time: 12min 4s


In [79]:
#解链
circle_ls = list(itertools.chain.from_iterable(circle_ls))

In [81]:
len(circle_ls)

2282

In [83]:
save_cache(circle_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/深圳_circle_ls.pickle")

In [115]:
circle_ls = read_cache("/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/深圳_circle_ls.pickle")
len(circle_ls)
circle_chunk = get_chunk(circle_ls,10)

In [116]:
len(circle_chunk)

229

In [ ]:
%%time
if __name__=='__main__':
    
    for index,chunk in enumerate(circle_chunk[62:]):
        offset_index = 62+index
        print("正在爬第{0}chunk".format(offset_index))
        pool = multiprocessing.Pool(processes=3) # 创建3个进程
        result = []
        for i in chunk:
            result.append(pool.apply_async(get_stores, (i, )))
        pool.close() # 关闭进程池，表示不能再往进程池中添加进程，需要在join之前调用
        pool.join() # 等待进程池中的所有进程执行完毕

        store_ls = []
        for res in result:
            store_ls.append(res.get())
        
        print("第{0}chunk已经爬完了".format(offset_index))
        store_ls = list(itertools.chain.from_iterable(store_ls))
        save_cache(store_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/深圳_store_ls_{0}_chunk.pickle".format(offset_index))
        time.sleep(60)
#     print("***********准备爬商家的详情信息******************")
#     pool = multiprocessing.Pool(processes=3) # 创建3个进程
#     result=[]
#     for i in store_ls:
#         result.append(pool.apply_async(get_info,(i,)))
#     pool.close()
#     pool.join()
    
#     print("信息页已经爬完了！！")
#     info_ls=[]
#     for res in result:
#         info_ls.append(res.get())
#     save_cache(info_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_info_test_ls.pickle")

正在爬第62chunk
正在爬取第1页
正在爬取第1页
正在爬取第1页
正在爬取第1页
正在爬取第1页
正在爬取第1页
正在爬取第2页
正在爬取第2页
正在爬取第2页
正在爬取第3页
正在爬取第3页
正在爬取第3页
正在爬取第4页
正在爬取第4页
正在爬取第4页
正在爬取第5页
正在爬取第5页
正在爬取第5页
正在爬取第1页
正在爬取第6页
正在爬取第6页
正在爬取第2页
正在爬取第7页
正在爬取第3页
正在爬取第8页
正在爬取第7页
正在爬取第4页
正在爬取第8页
正在爬取第9页
正在爬取第5页
正在爬取第1页
正在爬取第10页
正在爬取第6页
正在爬取第2页
正在爬取第11页
正在爬取第7页
正在爬取第3页
正在爬取第12页
正在爬取第13页
正在爬取第4页
正在爬取第8页
正在爬取第1页
正在爬取第14页
正在爬取第5页
正在爬取第1页
正在爬取第6页
正在爬取第2页
正在爬取第2页
正在爬取第7页
正在爬取第3页
正在爬取第3页
正在爬取第4页
正在爬取第8页
正在爬取第5页
正在爬取第6页
正在爬取第7页
正在爬取第8页
第62chunk已经爬完了
正在爬第63chunk
正在爬取第1页
正在爬取第1页
正在爬取第1页
正在爬取第1页
正在爬取第2页
正在爬取第2页
正在爬取第1页
正在爬取第3页
正在爬取第3页
正在爬取第2页
正在爬取第1页
正在爬取第4页
正在爬取第3页
正在爬取第2页
正在爬取第5页
正在爬取第4页
正在爬取第3页
正在爬取第6页
正在爬取第5页
正在爬取第4页
正在爬取第1页
正在爬取第6页
正在爬取第5页
正在爬取第2页
正在爬取第7页
正在爬取第6页
正在爬取第3页
正在爬取第8页
正在爬取第7页
正在爬取第4页
正在爬取第9页
正在爬取第8页
正在爬取第5页
正在爬取第10页
正在爬取第9页
正在爬取第6页
正在爬取第1页
正在爬取第7页
正在爬取第10页
正在爬取第1页
正在爬取第8页
正在爬取第11页
正在爬取第2页
正在爬取第9页
正在爬取第12页
正在爬取第3页
正在爬取第10页
正在爬取第13页
正在爬取第4页
正在爬取第1页
正在爬取第14页
正在爬取第5页
正在爬取第15页
正在爬取第6页
正在爬取第16页
正在爬取第17页
第63chunk已经爬完了
正在爬第64chunk
正在爬取第1页
正在爬取第1页
正

In [70]:
soup = requests_visit_url("http://www.dianping.com/search/category/4/10/g210r1519")#,["http","111.155.116.238:8123"])

In [90]:
a = read_cache("/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/深圳_store_ls.pickle")

In [ ]:
def foo(a):
    try:
        a = a+1
        print(a)
    except:
        raise MyException("访问被拒")
   

In [65]:
class MyException(Exception):
    def __init__(self,message):
        Exception.__init__(self)
        self.message=message

In [68]:
def main(a):
    try:
        foo(a)
    except MyException as e:
        